In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

2024-01-29 00:06:37.845991: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/hugo/tensorflow-metal-test/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
#Preprocessing --- Goal to make the data compatible for CNN input
# In tensorflow, we can achieve the same using ImageGenerators

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.)

In [3]:
#Pass the images through the generator
trainImageData = train_generator.flow_from_directory("../data/training",
                                                     batch_size=32, #how much images to give per iteration in an epoch
                                                     class_mode="binary", #Incase of multi-class classification, "categorical"
                                                     target_size=(64,64) #Ensures all images are of same size (resizing)
                                                     ) 


testImageData = train_generator.flow_from_directory("../data/testing",
                                                     batch_size=32, #how much images to give per iteration in an epoch
                                                     class_mode="binary", #Incase of multi-class classification, "categorical"
                                                     target_size=(64,64) #Ensures all images are of same size (resizing)
                                                     ) 

Found 2392 images belonging to 2 classes.
Found 597 images belonging to 2 classes.


In [4]:
#Given dataset is a balanced dataset

trainImageData.class_indices

{'flip': 0, 'notflip': 1}

In [6]:
trainImageData.image_shape

(64, 64, 3)

# Transferlearn from VGG16 - keras

In [16]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential

In [20]:
# Importing the VGG16 model but discarding the last 1000 neuron layer for adjustment to our dataset
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [21]:
# Freeze the base_model
base_model.trainable = False

In [23]:
# Create the model

model = tf.keras.models.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')  # Single neuron for binary classification
])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              525312    
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 15,241,025
Trainable params: 526,337
Non-trainable params: 14,714,688
_________________________________________________________________


In [26]:
#Compile

model.compile(optimizer="adam",
              loss="binary_crossentropy", #For multi-class classification: categorical_crossentropy | sparse_categorical_crossentropy
              metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

In [27]:
len(trainImageData.filenames) // trainImageData.batch_size

74

In [28]:
# Fit the model
model.fit(trainImageData,
          validation_data=testImageData,
          epochs=10,
          steps_per_epoch = len(trainImageData.filenames) // trainImageData.batch_size,
          validation_steps= len(testImageData.filenames) // testImageData.batch_size)

Epoch 1/10


2024-01-29 00:12:16.939401: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-29 00:12:17.410474: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-29 00:12:17.445905: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


74/74 [==============================] - ETA: 0s - loss: 0.5361 - f1_score: 0.7342

2024-01-29 00:12:48.860565: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-29 00:12:48.942856: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


74/74 [==============================] - 39s 500ms/step - loss: 0.5361 - f1_score: 0.7342 - val_loss: 0.5012 - val_f1_score: 0.8017
Epoch 2/10
74/74 [==============================] - 37s 505ms/step - loss: 0.3447 - f1_score: 0.8739 - val_loss: 0.3158 - val_f1_score: 0.8818
Epoch 3/10
74/74 [==============================] - 37s 507ms/step - loss: 0.2518 - f1_score: 0.9111 - val_loss: 0.2417 - val_f1_score: 0.9198
Epoch 4/10
74/74 [==============================] - 37s 508ms/step - loss: 0.2038 - f1_score: 0.9309 - val_loss: 0.2152 - val_f1_score: 0.9220
Epoch 5/10
74/74 [==============================] - 37s 507ms/step - loss: 0.1763 - f1_score: 0.9377 - val_loss: 0.2409 - val_f1_score: 0.9188
Epoch 6/10
74/74 [==============================] - 38s 508ms/step - loss: 0.1764 - f1_score: 0.9307 - val_loss: 0.1810 - val_f1_score: 0.9418
Epoch 7/10
74/74 [==============================] - 38s 508ms/step - loss: 0.1391 - f1_score: 0.9517 - val_loss: 0.2011 - val_f1_score: 0.9171
Epoch 8/10

In [30]:
model.save('../models/VGG16.keras')
#model.save('monReader.keras_tf', save_format='tf')

### The VGG16 re-trained to our dataset is 65MB, so it will NOT fit in a smartphone app! 